# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
params1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
params2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
params3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params3

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
# for testing mongodb model
# model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

# data parameters from model meta
params['ingestion_param_X']['feature_list'] = model_meta['ingestion_param_X']['feature_list']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']
params['data_y_flag'] = model_meta['data_y_flag']

# scaler/transform info for test data preparation
params['scaler_info'] = model_meta['scaler_param']
params['transform_info'] = model_meta['transform_param']

# model info for ML test
params['model_info'] = model_meta['model_info']

In [5]:
# 3. Data Preparation
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'classification_actionPattern_cleanLevel0_testX', 'feature_list': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']}


/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


window_size: 128 nan_limit_num: 64
(177280, 9) (1385, 128, 9)
(1385, 1) (1385, 1)


In [6]:
# set scaler by data y flag
if params['data_y_flag']:
    scaler = scaler_y
else:
    scaler = scaler_X

In [7]:
# model test
result = ML_api.ml_test(params, test_X_array, test_y_array, scaler)


Start testing data

** Performance of test dataset ==> PROB = [[9.88606513e-01 1.11198872e-02 6.66037813e-05 4.72046086e-05
  1.08898952e-04 5.09538295e-05]
 [9.88982856e-01 1.07509112e-02 6.49140202e-05 4.56086491e-05
  1.05853527e-04 4.97398942e-05]
 [9.88649249e-01 1.10789053e-02 6.60964433e-05 4.68110156e-05
  1.08175671e-04 5.06373726e-05]
 ...
 [9.88697648e-01 1.10319378e-02 6.57681230e-05 4.65326666e-05
  1.07591419e-04 5.04058298e-05]
 [9.89022493e-01 1.07125770e-02 6.46118788e-05 4.53484608e-05
  1.05442450e-04 4.95536333e-05]
 [9.88777101e-01 1.09533630e-02 6.56157135e-05 4.63267825e-05
  1.07247135e-04 5.02771763e-05]], ACC = 0.9803779069767442
** Dimension of result for test dataset = (1376,)
scale
=========================data
      value  prediction
0       0.0         0.0
1       0.0         0.0
2       0.0         0.0
3       0.0         0.0
4       0.0         0.0
...     ...         ...
1371    0.0         0.0
1372    0.0         0.0
1373    0.0         0.0
1374    0

In [8]:
result

{'result': {'index': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   

In [9]:
model_meta['model_info']

{'model_purpose': 'classification',
 'model_method': 'LSTM_cf',
 'model_name': 'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'input_size': 9,
  'seq_len': 128,
  'output_dim': 1,
  'num_classes': 6,
  'num_layers': 2,
  'hidden_size': 64,
  'dropout': 0.1,
  'bidirectional': True,
  'rnn_type': 'lstm'},
 'seq_len': 128,
 'input_size': 9,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/LSTM_cf/classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_/classification_actionPattern_cleanLevel0_trainX/model.pkl'}}}